Importation of packages

In [1]:
from filter_data import * # class containing filter methods 
from date_handling import * # class containing date related methods

Definition of constants 


In [2]:
START_DATE = np.datetime64('2014-07-01') # Date of the first published tweet in English 
END_DATE = np.datetime64('2018-06-01') # Date of the last published tweet in English 

RAW_DATA_PATH = "./data/original_data/IRAhandle_tweets_{}.csv"
PRE_PROCESSED_DATA_PATH = "./data/filtered_data/IRAhandle_tweets_{}.csv"


Filter the data : 
- Keep only tweets written in english 
- Remove links in the tweets
- Remove columns that we do not need 
- Reformat dates

In [3]:
# Define the set of column indices to keep
COLUMN_INDICES = {1, 2, 5, 13}

# Call the function with the input file name and column indices
for i in range(1, 14): 
    input_path= RAW_DATA_PATH.format(i)
    output_path= PRE_PROCESSED_DATA_PATH.format(i)
    filter_csv(input_path, output_path, COLUMN_INDICES, 4, "English") # Keep only tweets in english 
    extract_date_columns(output_path, output_path)
    remove_links(output_path, output_path)
    convert_to_lowercase(output_path, output_path)